# 🔥 FILE 2-C: CNN & Model Checkpoint

**PHẦN 2 - INTERMEDIATE (CORE DEEP LEARNING) - FINAL**

---

## 📋 Nội Dung

✅ CNN (Convolutional Neural Networks) là gì

✅ Conv2d - Convolutional layers

✅ Pooling - MaxPool, AvgPool

✅ CNN Architecture patterns

✅ Dự án: Image Classification với CNN

✅ Save & Load model checkpoints

✅ Best practices cho model saving

---

## ⏱️ Thời Gian Học: 3-4 giờ

---

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

---

# 1️⃣ CNN Cơ Bản

## Tại Sao Cần CNN?

**Vấn đề với Fully Connected cho ảnh:**

```
Ảnh 224×224×3 (RGB)
→ Flatten: 224 × 224 × 3 = 150,528 features
→ FC(150528, 1000): 150M+ parameters!
```

❌ Quá nhiều parameters

❌ Mất spatial information

❌ Không translation invariant

**CNN giải quyết:**

✅ Local connectivity (receptive field)

✅ Parameter sharing (same filter)

✅ Translation invariance

✅ Hierarchical features

## Conv2d Layer

In [ ]:
print("=" * 70)
print("nn.Conv2d")
print("=" * 70)

# Conv2d parameters
conv = nn.Conv2d(
    in_channels=3,      # Input channels (RGB = 3)
    out_channels=16,    # Output channels (number of filters)
    kernel_size=3,      # Filter size: 3×3
    stride=1,           # Stride
    padding=1           # Padding
)

print(f"Conv layer: {conv}")
print(f"\nParameters:")
print(f"  Weight shape: {conv.weight.shape}")
print(f"  → (out_channels, in_channels, kernel_h, kernel_w)")
print(f"  Bias shape: {conv.bias.shape}")
print(f"  Total params: {conv.weight.numel() + conv.bias.numel()}")

# Test forward
x = torch.randn(1, 3, 32, 32)  # (batch, channels, height, width)
out = conv(x)
print(f"\nInput: {x.shape}")
print(f"Output: {out.shape}")

print("""
CÔNG THỨC OUTPUT SIZE:

H_out = (H_in + 2*padding - kernel_size) / stride + 1
W_out = (W_in + 2*padding - kernel_size) / stride + 1

Ví dụ:
  Input: 32×32
  kernel=3, padding=1, stride=1
  Output: (32 + 2*1 - 3)/1 + 1 = 32×32

💡 SAME PADDING: padding = (kernel_size - 1) / 2
   → Output size = Input size (when stride=1)
""")

## Pooling Layers

In [ ]:
print("=" * 70)
print("POOLING LAYERS")
print("=" * 70)

# MaxPool
maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

# AvgPool
avgpool = nn.AvgPool2d(kernel_size=2, stride=2)

x = torch.randn(1, 16, 32, 32)
max_out = maxpool(x)
avg_out = avgpool(x)

print(f"Input: {x.shape}")
print(f"MaxPool output: {max_out.shape}")
print(f"AvgPool output: {avg_out.shape}")

print("""
POOLING:

MaxPool2d(2, 2):
  - Chia thành windows 2×2
  - Lấy giá trị MAX trong mỗi window
  - Giảm size: 32×32 → 16×16

AvgPool2d(2, 2):
  - Lấy TRUNG BÌNH thay vì max

LỢI ÍCH:
  ✅ Downsampling (giảm spatial size)
  ✅ Giảm parameters
  ✅ Translation invariance
  ✅ Tăng receptive field

KHUYẾN NGHỊ:
  - MaxPool: Phổ biến hơn
  - kernel=2, stride=2: Standard
""")

---

# 2️⃣ CNN Architecture

In [ ]:
print("=" * 70)
print("SIMPLE CNN ARCHITECTURE")
print("=" * 70)

class SimpleCNN(nn.Module):
    """Simple CNN for image classification"""
    
    def __init__(self, num_classes=10):
        super().__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Conv block 1: 32×32 → 16×16
        x = self.pool(torch.relu(self.conv1(x)))
        
        # Conv block 2: 16×16 → 8×8
        x = self.pool(torch.relu(self.conv2(x)))
        
        # Conv block 3: 8×8 → 4×4
        x = self.pool(torch.relu(self.conv3(x)))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x

model = SimpleCNN(num_classes=10)
print(model)

# Test
x = torch.randn(1, 3, 32, 32)
out = model(x)
print(f"\nInput: {x.shape}")
print(f"Output: {out.shape}")
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## Modern CNN Pattern

In [ ]:
print("=" * 70)
print("MODERN CNN WITH BATCHNORM")
print("=" * 70)

class ModernCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        
        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(3, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 2
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Block 3
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

print("""
MODERN CNN PATTERN:

Conv → BatchNorm → ReLU → Conv → BatchNorm → ReLU → Pool

KEY POINTS:
  ✅ BatchNorm after Conv
  ✅ Double Conv before pooling
  ✅ Increasing channels: 64 → 128 → 256
  ✅ Decreasing spatial: 32 → 16 → 8 → 4
  ✅ Dropout in FC layers
""")

---

# 3️⃣ Dự Án: Image Classification

## Load CIFAR-10 Dataset

In [ ]:
print("=" * 70)
print("LOADING CIFAR-10")
print("=" * 70)

# Data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Train samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Classes: {classes}")

# Visualize samples
dataiter = iter(train_loader)
images, labels = next(dataiter)

fig, axes = plt.subplots(2, 8, figsize=(16, 4))
for i, ax in enumerate(axes.flatten()):
    img = images[i].numpy().transpose(1, 2, 0)
    img = img * 0.5 + 0.5  # Denormalize
    ax.imshow(img)
    ax.set_title(classes[labels[i]])
    ax.axis('off')
plt.tight_layout()
plt.show()

## Training CNN

In [ ]:
print("=" * 70)
print("TRAINING CNN")
print("=" * 70)

# Model
model = SimpleCNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
num_epochs = 10
history = {'train_loss': [], 'train_acc': [], 'test_acc': []}

for epoch in range(num_epochs):
    # Train
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total
    
    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_acc = 100 * correct / total
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {train_loss:.3f} "
          f"Train Acc: {train_acc:.1f}% Test Acc: {test_acc:.1f}%")

print("\n✅ Training completed!")

---

# 4️⃣ Save & Load Model

## Cách Lưu Model

In [ ]:
print("=" * 70)
print("SAVE & LOAD MODEL")
print("=" * 70)

# CÁCH 1: Save toàn bộ model (KHÔNG khuyến khích)
torch.save(model, 'entire_model.pth')
loaded_model = torch.load('entire_model.pth')
print("✓ Saved entire model")

# CÁCH 2: Save state_dict (KHUYẾN KHÍCH)
torch.save(model.state_dict(), 'model_weights.pth')
print("✓ Saved state_dict")

# Load state_dict
new_model = SimpleCNN(num_classes=10)
new_model.load_state_dict(torch.load('model_weights.pth'))
new_model.to(device)
print("✓ Loaded state_dict")

# CÁCH 3: Save checkpoint (BEST PRACTICE)
checkpoint = {
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': history['train_loss'][-1],
    'accuracy': history['test_acc'][-1],
}

torch.save(checkpoint, 'checkpoint.pth')
print("✓ Saved checkpoint")

# Load checkpoint
checkpoint = torch.load('checkpoint.pth')
model = SimpleCNN(num_classes=10)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer = optim.Adam(model.parameters())
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

print(f"✓ Loaded checkpoint from epoch {epoch}")

print("""

BEST PRACTICES:

1. Lưu state_dict, KHÔNG lưu toàn bộ model
2. Lưu optimizer state để resume training
3. Lưu epoch, loss, accuracy
4. Lưu hyperparameters, config
5. Đặt tên file rõ ràng: model_epoch10_acc85.pth

CHECKPOINT STRUCTURE:
{
    'epoch': int,
    'model_state_dict': OrderedDict,
    'optimizer_state_dict': dict,
    'loss': float,
    'accuracy': float,
    'config': dict  # Optional
}
""")

## Early Stopping với Checkpoint

In [ ]:
print("=" * 70)
print("TRAINING WITH CHECKPOINTING")
print("=" * 70)

class ModelCheckpoint:
    """Save best model during training"""
    
    def __init__(self, filepath, monitor='val_loss', mode='min', verbose=True):
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.verbose = verbose
        self.best = float('inf') if mode == 'min' else float('-inf')
    
    def __call__(self, model, optimizer, epoch, metrics):
        current = metrics[self.monitor]
        
        if self.mode == 'min':
            is_better = current < self.best
        else:
            is_better = current > self.best
        
        if is_better:
            self.best = current
            
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                **metrics
            }
            
            torch.save(checkpoint, self.filepath)
            
            if self.verbose:
                print(f"\n💾 Saved checkpoint: {self.monitor}={current:.4f}")

print("""
USAGE:

checkpoint = ModelCheckpoint(
    filepath='best_model.pth',
    monitor='val_acc',
    mode='max'
)

for epoch in range(epochs):
    # Training...
    metrics = {
        'train_loss': train_loss,
        'val_loss': val_loss,
        'val_acc': val_acc
    }
    checkpoint(model, optimizer, epoch, metrics)
""")

---

# ✅ Tổng Kết FILE 2-C & INTERMEDIATE LEVEL

## FILE 2-C: Học Được

✅ **CNN basics**: Conv2d, Pooling

✅ **CNN architecture**: Pattern thiết kế

✅ **Image classification**: CIFAR-10 project

✅ **Model checkpoint**: Save/load best practices

---

## 🎉 HOÀN THÀNH INTERMEDIATE LEVEL!

### Tổng Kết PHẦN 2:

📗 **FILE 2-A**: Dataset & DataLoader
- Custom Dataset
- DataLoader
- Training/Validation loop
- Overfitting/Underfitting
- Early Stopping

📗 **FILE 2-B**: Optimizer & Regularization
- Activation functions
- Optimizers (SGD, Adam, AdamW)
- Learning rate scheduling
- Regularization (Dropout, Weight Decay, BatchNorm)

📗 **FILE 2-C**: CNN & Checkpoint
- CNN architecture
- Image classification
- Model saving/loading

---

## Bạn Giờ Có Thể:

✅ Xử lý data với Dataset/DataLoader

✅ Build training pipeline hoàn chỉnh

✅ Detect và fix overfitting

✅ Tune hyperparameters

✅ Build CNN cho image tasks

✅ Save/load models đúng cách

---

## Template: Complete CNN Training Pipeline

```python
# 1. Data
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

# 2. Model
model = ModernCNN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# 3. Training
checkpoint = ModelCheckpoint('best_model.pth', monitor='val_acc', mode='max')
early_stop = EarlyStopping(patience=10)

for epoch in range(epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, ...)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, ...)
    
    # Save best
    checkpoint(model, optimizer, epoch, {'val_acc': val_acc, ...})
    
    # Early stopping
    early_stop(val_loss)
    if early_stop.early_stop:
        break
    
    # Update LR
    scheduler.step()
```

---

## 🚀 Bước Tiếp Theo

Bạn đã sẵn sàng cho:
- Transfer Learning
- Advanced architectures (ResNet, VGG, etc.)
- Object Detection
- Semantic Segmentation
- GANs, Transformers

**Congratulations on completing Intermediate level! 🎉**

---